In [ ]:
            from refactor.generator.model import RetrievalAugmentedGenerator

ckpt_path = '../gen.ckpt'
tac_gen = RetrievalAugmentedGenerator.load(
    ckpt_path, device='cuda', freeze=False
)

In [ ]:
from refactor.process_traces import get_traces

trace = get_traces('../experiments/runs/eval_loop/bestfs_2023_11_16/07_59_20/traces/abs_sub_round_eq_min')[0]

In [ ]:
def reconstruct_scores(state, sequence):
    tokenized_sequence= tac_gen.tokenizer(
        sequence,
        padding="longest",
        max_length=2300,
        truncation=True,
        return_tensors="pt",
    )

    tokenized_state = tac_gen.tokenizer(
        state,
        padding="longest",
        max_length=2300,
        truncation=True,
        return_tensors="pt",
    )

    state_ids = tokenized_state.input_ids.cuda()
    state_mask = tokenized_state.attention_mask.cuda()

    targets = tokenized_sequence.input_ids.cuda()

    targets[targets == tac_gen.tokenizer.pad_token_id] = -100


    output = tac_gen.generator(
        input_ids=state_ids,
        attention_mask=state_mask,
        labels=targets
    ).logits

    return output, tokenized_sequence.input_ids.cuda()

In [ ]:
edge = trace.tree.out_edges[0]

In [ ]:
tac = edge.tactic
logprob = edge.tac_logprob

In [ ]:
trace.tree.out_edges[:2]

In [ ]:
import torch
tacs = ['cases <a>le_total</a> x (1 - <a>round</a> x) with hx hx', 'cases <a>le_total</a> x 1 with hx hx']

In [ ]:
logits, seq_ids = reconstruct_scores([trace.tree.goal, trace.tree.goal], tacs)

In [ ]:
normalised = torch.log_softmax(logits, -1)

In [ ]:
outs = torch.gather(normalised, -1, seq_ids.unsqueeze(-1)).squeeze(-1)


In [ ]:
outs[seq_ids == tac_gen.tokenizer.pad_token_id] = 0

In [ ]:
torch.sum(outs, dim=1)